In [5]:
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-"

In [2]:
# !pip install langchain faiss-cpu huggingface_hub sentence-transformers

In [1]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [2]:
from langchain.schema import Document

# Create LangChain documents for IPL players

doc1 = Document(
        page_content="Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.",
        metadata={"team": "Royal Challengers Bangalore"}
    )
doc2 = Document(
        page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
        metadata={"team": "Mumbai Indians"}
    )
doc3 = Document(
        page_content="MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.",
        metadata={"team": "Chennai Super Kings"}
    )
doc4 = Document(
        page_content="Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.",
        metadata={"team": "Mumbai Indians"}
    )
doc5 = Document(
        page_content="Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.",
        metadata={"team": "Chennai Super Kings"}
    )


In [3]:
docs = [doc1, doc2, doc3, doc4, doc5]

In [10]:
# vector_store = Chroma(
#     embedding_function=OpenAIEmbeddings(),
#     persist_directory='11.langchain-chroma\my_chroma_db',
#     collection_name='sample'
# )

# Initialize embedding model
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Save
vector_store = FAISS.from_documents(docs, embedding)
vector_store.save_local("my_faiss_db")

# Load
# vector_store = FAISS.load_local("my_faiss_db", embedding) #Load

In [11]:
# # add documents
vector_store.add_documents(docs)

['3ad2374d-ef9c-484c-9f20-53bc1fab78ce',
 'fdb6abed-78ee-4242-915f-04f4fbbc8292',
 'd4153ef5-65ec-4dff-b4de-2106a0803e74',
 'c7db2059-d945-4671-819e-53a5131bd823',
 'c2a2b46c-231d-4834-8778-6572080b6f38']

In [12]:
# view documents
# vector_store.get(include=['embeddings','documents', 'metadatas'])

stored_docs = vector_store.docstore._dict.values()
for doc in stored_docs:
    print("Content:", doc.page_content)
    print("Metadata:", doc.metadata)
    print("-" * 50)

Content: Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.
Metadata: {'team': 'Royal Challengers Bangalore'}
--------------------------------------------------
Content: Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.
Metadata: {'team': 'Mumbai Indians'}
--------------------------------------------------
Content: MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.
Metadata: {'team': 'Chennai Super Kings'}
--------------------------------------------------
Content: Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over

In [17]:
# search documents
vector_store.similarity_search(
    query='Who among these are a baowler?',
    k=2
)

[Document(id='495d1505-b8a2-4869-94ee-191be525e7ae', metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
 Document(id='c7db2059-d945-4671-819e-53a5131bd823', metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.')]

In [20]:
# search with similarity score
vector_store.similarity_search_with_score(
    query='Who among these are a batter?',
    k=3
)

[(Document(id='779b3266-66f6-4102-b5c6-233f26356463', metadata={'team': 'Mumbai Indians'}, page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure."),
  np.float32(1.3784537)),
 (Document(id='fdb6abed-78ee-4242-915f-04f4fbbc8292', metadata={'team': 'Mumbai Indians'}, page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure."),
  np.float32(1.3784537)),
 (Document(id='feffec67-58e9-4db9-93b5-995bff73c455', metadata={'team': 'Royal Challengers Bangalore'}, page_content='Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.'),
  np.float32(1.4477019))]

In [15]:
# meta-data filtering
vector_store.similarity_search_with_score(
    query="",
    filter={"team": "Chennai Super Kings"}
)

[(Document(id='94b2fbd6-8355-43f8-8550-feacf866d6a7', metadata={'team': 'Chennai Super Kings'}, page_content='MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.'),
  np.float32(1.8436006)),
 (Document(id='d4153ef5-65ec-4dff-b4de-2106a0803e74', metadata={'team': 'Chennai Super Kings'}, page_content='MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.'),
  np.float32(1.8436006)),
 (Document(id='802272bd-6bb2-49fd-83b7-d2bd0353bb97', metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.'),
  np.float32(1.8909373)),
 (Document(id='c2a2b46c-231d-4834-8778-6572080b6f38', metadata={'team': '

In [22]:
# # update documents
# updated_doc1 = Document(
#     page_content="Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple centuries in a single season. Despite RCB not winning an IPL title under his captaincy, Kohli's passion and fitness set a benchmark for the league. His ability to chase targets and anchor innings has made him one of the most dependable players in T20 cricket.",
#     metadata={"team": "Royal Challengers Bangalore"}
# )

# vector_store.update_document(document_id='feffec67-58e9-4db9-93b5-995bff73c455', document=updated_doc1)


In [13]:
# view documents
stored_docs = list(vector_store.docstore._dict.values())

for doc in stored_docs:
    print("Content:", doc.page_content)
    print("Metadata:", doc.metadata)
    print("-" * 40)

Content: Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.
Metadata: {'team': 'Royal Challengers Bangalore'}
----------------------------------------
Content: Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.
Metadata: {'team': 'Mumbai Indians'}
----------------------------------------
Content: MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.
Metadata: {'team': 'Chennai Super Kings'}
----------------------------------------
Content: Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.
Metadata: {'team':

In [15]:
# delete document
# vector_store.delete(ids=['09a39dc6-3ba6-4ea7-927e-fdda591da5e4'])

In [16]:
# view documents
# vector_store.get(include=['embeddings','documents', 'metadatas'])

docs = list(vector_store.docstore._dict.values())

# Get all embeddings (as numpy array)
embeddings = vector_store.index.reconstruct_n(0, vector_store.index.ntotal)

# View document content, metadata, and embeddings together
for i, doc in enumerate(docs):
    print(f"Document {i+1}")
    print("Content:", doc.page_content)
    print("Metadata:", doc.metadata)
    print("Embedding:", embeddings[i])
    print("-" * 50)

Document 1
Content: Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.
Metadata: {'team': 'Royal Challengers Bangalore'}
Embedding: [ 9.94725246e-03  6.91433921e-02 -5.14711514e-02 -3.45005319e-02
 -2.83650425e-03  6.90014437e-02  4.55914587e-02  2.21575629e-02
 -2.20000315e-02 -2.23901533e-02 -2.40122266e-02 -7.26742968e-02
  6.67429194e-02  8.84867162e-02  9.64311138e-02  3.35256686e-03
  4.33897898e-02  5.54265119e-02  1.81250311e-02 -5.61771765e-02
 -7.00142458e-02  8.38762820e-02  6.82230573e-03 -3.15809101e-02
 -1.25386687e-02 -9.54672694e-03 -2.30111722e-02  6.15350418e-02
  7.90988654e-03 -8.23711455e-02  5.22753336e-02 -1.91612225e-02
 -4.71369401e-02  2.11308617e-02 -4.25533541e-02 -4.62207245e-03
 -6.79354891e-02  9.03680250e-02  1.53079536e-02 -4.81577069e-02
 -1.55495219e-02  1.94462389e-02 -2.21338756e-02 -2.16198955e-02
 -1.434